In [1]:
import sys
import pandas as pd
import numpy as np
import math

In [ ]:
## 修改讀取方式
## 1. 加常數項

## Read data

In [2]:
def Read_Data():
    df = pd.read_csv("../data/train.csv", encoding='big5')
    df = df.iloc[:, 3:]
    df[df == 'NR'] = 0
    df[df.columns] = df[df.columns].astype(float)
    train_data = df.to_numpy()
    
    
    month_data = {}
    for month in range(12):
        sample = np.empty([18,480])
        for day in range(20):
            sample[:, day*24 : (day+1)*24] = train_data[18*(20*month + day) : 18*(20*month + day + 1), :]

        month_data[month] = sample

    x = np.empty([12 * 471, 18 * 9], dtype = float)
    y = np.empty([12 * 471, 1], dtype = float)
    for month in range(12):
        for day in range(20):
            for hour in range(24):
                if day == 19 and hour > 14:
                    continue
                x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) 
                y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]

    mean_x = np.mean(x, axis = 0) #18 * 9 
    std_x = np.std(x, axis = 0) #18 * 9 
    for i in range(len(x)): #12 * 471
        for j in range(len(x[0])): #18 * 9 
            if std_x[j] != 0:
                x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
    
    return x

In [5]:
a = np.asarray([[1,2],[3,4]])
b = np.asarray([1,2])
np.dot(a,b)

array([ 5, 11])

In [ ]:
def K_Fold(X, n_parts):
    pass

## Linear Regression

In [8]:
class LinearRegressionUsingGD:
    
    def __init__(self, lr=0.05, iters=1000):
        self.lr = lr
        self.iters = iters

    def fit(self, x, y):
        self.cost_ = []
        self.w_ = np.zeros((x.shape[1], 1))
        m = x.shape[0]

        for _ in range(self.iters):
            y_pred = np.dot(x, self.w_)
            residuals = y_pred - y
            gradient_vector = np.dot(x.T, residuals)
            self.w_ -= (self.lr / m) * gradient_vector
            cost = np.sum((residuals ** 2)) / (2 * m)
            self.cost_.append(cost)
            
        return self

    def predict(self, x):
        return np.dot(x, self.w_)

## Training

In [10]:
## 前製作業
N_FOLDS = 5

X = Read_Data()
k_fold = K_Fold(X, N_FOLDS)

weight_best = None
loss_best = sys.maxint
for train_idx, valid_idx in k_fold:
    x_train = x[train_idx, :]
    y_train = y[train_idx, :]
    x_valid = x[valid_idx, :]
    y_valid = y[valid_idx, :]
    
    LRGD = LinearRegressionUsingGD()
    weight = LRGD.fit(x_train, y_train)
    predict = LRGD.predict(x_valid)
    loss = np.sum((y_valid - predict)**2) ## MSE
    
    if(loss < loss_best):
        weight_best = weight
        loss_best = loss